In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from catboost import Pool
from catboost import CatBoostClassifier
from catboost.utils import get_roc_curve, get_confusion_matrix, select_threshold

# Легенда

Вернемся к гномам - исследователям в сфере Машинного Обучения! 

Гномы достаточно дружные, искренние и доверчивые существа и живут общинами. В каждом поселении есть таверна, в которой разливают вкусный эль и пекут замечательные булочки. Не смотря на все свои светлые качества, гномы достаточно забывчивые и расторопные, поэтому после завершения домашних дел перед ужином в таверне забывают кошелек, а расплачиваются утром на следующий день.

Особо нечестные жители общины могут и вовсе не вернуть деньги заведению, поэтому любители гномьего Машинного Обучения предложили следующую идею: научиться предсказывать, вернет ли забывший дома кошелек гном деньги в течение ближайших дней или, на современном языке, его заказ дефолтнет. Это позволит повысить финансовую стабильность таверны, не исполняя заказы, которые заранее не будут оплачены.

Задача - построить алгоритм, определяющий вероятность дефолта того или иного заказа на исторических данных из таверн различных гномьих общин с 2015-02-24 по 2016-09-30.

In [2]:
dwarves = pd.read_csv("../data/raw/train.csv")
dwarves.head()

,Deal_id,Deal_date,First_deal_date,Secret_dwarf_info_1,Secret_dwarf_info_2,Secret_dwarf_info_3,First_default_date,Successful_deals_count,Region,Tavern,Hashed_deal_detail_1,Hashed_deal_detail_2,Hashed_deal_detail_3,Hashed_deal_detail_4,Hashed_deal_detail_5,Hashed_deal_detail_6,Age,Gender,Default
0,22487461,2015-11-05,2015-08-29,NaN,NaN,NaN,NaN,0.0,Tavern_district_3,7,2.5,-3,8,2.5,-3,5,36.0,Male,0
1,62494261,2016-08-26,2015-12-21,3.5,-2.0,5.0,2016-07-30,2.0,Tavern_district_4,7,2.5,-3,14,3.5,-3,5,29.0,Female,1
2,34822849,2016-02-18,2015-11-11,NaN,NaN,NaN,NaN,0.0,Tavern_district_6,7,2.5,-3,8,2.5,-3,5,56.0,Female,0
3,46893387,2016-04-30,2016-03-22,NaN,NaN,NaN,NaN,0.0,Tavern_district_2,13,2.5,-2,5,2.5,-3,5,27.0,Female,0
4,67128275,2016-09-19,2016-07-21,NaN,NaN,NaN,NaN,0.0,Tavern_district_4,39,2.5,-3,7,2.5,-3,5,37.0,Female,0


In [3]:
dwarves.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Deal_id                 3168 non-null   int64  
 1   Deal_date               3168 non-null   object 
 2   First_deal_date         3168 non-null   object 
 3   Secret_dwarf_info_1     535 non-null    float64
 4   Secret_dwarf_info_2     535 non-null    float64
 5   Secret_dwarf_info_3     535 non-null    float64
 6   First_default_date      535 non-null    object 
 7   Successful_deals_count  3154 non-null   float64
 8   Region                  3161 non-null   object 
 9   Tavern                  3168 non-null   int64  
 10  Hashed_deal_detail_1    3168 non-null   float64
 11  Hashed_deal_detail_2    3168 non-null   int64  
 12  Hashed_deal_detail_3    3168 non-null   int64  
 13  Hashed_deal_detail_4    3168 non-null   float64
 14  Hashed_deal_detail_5    3168 non-null   

In [4]:
dwarves.shape

(3168, 19)

- Очень мало данных. Либо разделяющая гиперплоскость слишком очевидна, либо будут проблемы (конечно же второе).

- Есть пустые значения в засекреченных признаках ('Secret_dwarf_info_n), процент пустых строк больше 83%.
- Признак 'First_default_date' (Первая дата дефолта гнома) интуитивно говорит нам,
что это подходит под самый ключевой признак,
(ведь закон "завтра будет то же, что и вчера" никто не отменял, NaN приведем к дате).
В дальнейшем используем его при генерации новых фичей.

- В качестве baseline мы "практически пустые" признаки удалим, но затем рассмотрим вариант с заполнением медианой.
- В фиче же "Successful_deals_count" и "Region" заполним пропуски "0" или самым популярным.
- Deal_id ('Номер заказа') очевидно бессмысленный для обучения признак.

# Боримся с выбросами

In [5]:
dwarves['Successful_deals_count'].value_counts()
# тут очевидно, заполним "0", по смыслу может быть интерпретировано
# как отсутсвие оплаченных заказов ранее
# или проще: что достоверных данных нет

0.0     1903
2.0      592
3.0      244
1.0      194
4.0      105
5.0       52
6.0       23
7.0       13
8.0        9
9.0        8
10.0       6
17.0       2
12.0       2
11.0       1
Name: Successful_deals_count, dtype: int64

In [6]:
dwarves['Region'].value_counts()
# для 7 пропусков разумно заполнить самым популярным.
# Также заметим, что с таким объемом датасета подойдет one-hot encoding данного признака.

Tavern_district_3    1204
Tavern_district_6     478
Tavern_district_2     448
Tavern_district_4     364
Tavern_district_1     240
Tavern_district_0     213
Tavern_district_5     160
Tavern_district_7      54
Name: Region, dtype: int64

In [7]:
dwarves['Successful_deals_count'] = dwarves['Successful_deals_count'].fillna(0)
dwarves['Region'] = dwarves['Region'].fillna('Tavern_district_3')
dwarves.head()

,Deal_id,Deal_date,First_deal_date,Secret_dwarf_info_1,Secret_dwarf_info_2,Secret_dwarf_info_3,First_default_date,Successful_deals_count,Region,Tavern,Hashed_deal_detail_1,Hashed_deal_detail_2,Hashed_deal_detail_3,Hashed_deal_detail_4,Hashed_deal_detail_5,Hashed_deal_detail_6,Age,Gender,Default
0,22487461,2015-11-05,2015-08-29,NaN,NaN,NaN,NaN,0.0,Tavern_district_3,7,2.5,-3,8,2.5,-3,5,36.0,Male,0
1,62494261,2016-08-26,2015-12-21,3.5,-2.0,5.0,2016-07-30,2.0,Tavern_district_4,7,2.5,-3,14,3.5,-3,5,29.0,Female,1
2,34822849,2016-02-18,2015-11-11,NaN,NaN,NaN,NaN,0.0,Tavern_district_6,7,2.5,-3,8,2.5,-3,5,56.0,Female,0
3,46893387,2016-04-30,2016-03-22,NaN,NaN,NaN,NaN,0.0,Tavern_district_2,13,2.5,-2,5,2.5,-3,5,27.0,Female,0
4,67128275,2016-09-19,2016-07-21,NaN,NaN,NaN,NaN,0.0,Tavern_district_4,39,2.5,-3,7,2.5,-3,5,37.0,Female,0


# Преобразование признаков и генерация новых

In [8]:
# one-hot encoding для Region. Тут мы сразу убрали 1ый столбец, чтобы избежать мультиколлинеарности.
dwarves = dwarves.join(pd.get_dummies(dwarves['Region'], drop_first=True))
dwarves['Gender'] = dwarves['Gender'].apply(lambda x: 0 if x == 'Female' else 1)

Хотим узнать, через сколько дней был первый дефолт (насколько быстро гном стал "ненадежным")?

In [9]:
# приведем к типу "дата"
dwarves['First_default_date'] = pd.to_datetime(dwarves['First_default_date'])
dwarves['First_deal_date'] = pd.to_datetime(dwarves['First_deal_date'])
dwarves['Deal_date'] = pd.to_datetime(dwarves['Deal_date'])


# если N/A для даты дефолта - примем как отсутствие дефолтов и заполним датой заказа.
dwarves['First_default_date'] = dwarves['First_default_date'].fillna(dwarves['Deal_date'])

# найдем разницу между днями от первого заказа до первого дефолта
# (в 'First_default_date' стоит день заказа, если дефолта не было)
dwarves['day_before_first_defolt'] = (dwarves['First_default_date'] - dwarves['First_deal_date']).dt.days

# предположим, что день месяца, месяц, is_weekend (выходной: да / нет) даты заказа ('Deal_date') имеют влияние на таргет
dwarves['Deal_day'] = dwarves['Deal_date'].dt.day
dwarves['Deal_month'] = dwarves['Deal_date'].dt.month
dwarves['is_weekend'] = dwarves['Deal_date'].dt.dayofweek.apply(lambda x: 1 if x>4 else 0)

In [10]:
dwarves.corr()['Default'] # видим, что что-то не так с 'Hashed_deal_detail_6'.

C:\Users\Dmitry\AppData\Local\Temp\ipykernel_6812\1316345619.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  dwarves.corr()['Default'] # видим, что что-то не так с 'Hashed_deal_detail_6'.


Deal_id                   -0.008057
Secret_dwarf_info_1        0.063776
Secret_dwarf_info_2        0.103164
Secret_dwarf_info_3        0.063684
Successful_deals_count    -0.079221
Tavern                    -0.030804
Hashed_deal_detail_1       0.023913
Hashed_deal_detail_2      -0.016287
Hashed_deal_detail_3      -0.065219
Hashed_deal_detail_4      -0.070956
Hashed_deal_detail_5       0.014640
Hashed_deal_detail_6            NaN
Age                       -0.125430
Gender                     0.073819
Default                    1.000000
Tavern_district_1         -0.032650
Tavern_district_2         -0.045128
Tavern_district_3          0.047240
Tavern_district_4          0.039957
Tavern_district_5         -0.007932
Tavern_district_6         -0.030794
Tavern_district_7         -0.023175
day_before_first_defolt   -0.168160
Deal_day                  -0.001661
Deal_month                 0.009004
is_weekend                -0.038824
Name: Default, dtype: float64

In [11]:
dwarves['Hashed_deal_detail_6'].value_counts() # одно значение во всем датасете, удаляем признак

5    3168
Name: Hashed_deal_detail_6, dtype: int64

In [12]:
# удалим "NaN" признаки, оригинал для one-hot encoding, номер заказа (идентификатор)
dwarves = dwarves.drop(columns=[
    'Secret_dwarf_info_1',
    'Secret_dwarf_info_2',
    'Secret_dwarf_info_3',
    'Deal_id',
    'Region',
    'Deal_date',
    'First_deal_date',
    'First_default_date',
    'Hashed_deal_detail_6'
])
dwarves.head()

,Successful_deals_count,Tavern,Hashed_deal_detail_1,Hashed_deal_detail_2,Hashed_deal_detail_3,Hashed_deal_detail_4,Hashed_deal_detail_5,Age,Gender,Default,...,Tavern_district_2,Tavern_district_3,Tavern_district_4,Tavern_district_5,Tavern_district_6,Tavern_district_7,day_before_first_defolt,Deal_day,Deal_month,is_weekend
0,0.0,7,2.5,-3,8,2.5,-3,36.0,1,0,...,0,1,0,0,0,0,68,5,11,0
1,2.0,7,2.5,-3,14,3.5,-3,29.0,0,1,...,0,0,1,0,0,0,222,26,8,0
2,0.0,7,2.5,-3,8,2.5,-3,56.0,0,0,...,0,0,0,0,1,0,99,18,2,0
3,0.0,13,2.5,-2,5,2.5,-3,27.0,0,0,...,1,0,0,0,0,0,39,30,4,1
4,0.0,39,2.5,-3,7,2.5,-3,37.0,0,0,...,0,0,1,0,0,0,60,19,9,0


In [13]:
dwarves.isna().sum().sum()

0

Отлично, пустых ячеек нет, датасет для baseline готов

In [14]:
# сохраним датасет в качестве baseline
dwarves.to_csv("../data/interim/dwarves_baseline.csv", index=False)

In [15]:
dwarves = pd.read_csv("../data/interim/dwarves_baseline.csv")
dwarves.head()

,Successful_deals_count,Tavern,Hashed_deal_detail_1,Hashed_deal_detail_2,Hashed_deal_detail_3,Hashed_deal_detail_4,Hashed_deal_detail_5,Age,Gender,Default,...,Tavern_district_2,Tavern_district_3,Tavern_district_4,Tavern_district_5,Tavern_district_6,Tavern_district_7,day_before_first_defolt,Deal_day,Deal_month,is_weekend
0,0.0,7,2.5,-3,8,2.5,-3,36.0,1,0,...,0,1,0,0,0,0,68,5,11,0
1,2.0,7,2.5,-3,14,3.5,-3,29.0,0,1,...,0,0,1,0,0,0,222,26,8,0
2,0.0,7,2.5,-3,8,2.5,-3,56.0,0,0,...,0,0,0,0,1,0,99,18,2,0
3,0.0,13,2.5,-2,5,2.5,-3,27.0,0,0,...,1,0,0,0,0,0,39,30,4,1
4,0.0,39,2.5,-3,7,2.5,-3,37.0,0,0,...,0,0,1,0,0,0,60,19,9,0


In [16]:
dwarves['Default'].value_counts()

# Еще одна назревшая проблема. Таргет несбалансирован. Отложим это в уме.

0    2817
1     351
Name: Default, dtype: int64

Используя различные классификаторы:
* svm.SVC(kernel="rbf", C=30000, random_state=2023)
* CatBoostClassifier(l2_leaf_reg=10, depth=3, learning_rate=0.9, iterations=1000, random_seed=2023)
* RandomForestClassifier()

Убеждаемся, что находимся практически на уровне со случайным угадыванием. Так еще и с очень низким recall.

Рассмотрим на примере Catboost-а.

# Приведение типов под catboost

In [17]:
X = dwarves.drop(columns=['Default'])
y = dwarves['Default']

In [18]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Successful_deals_count   3168 non-null   float64
 1   Tavern                   3168 non-null   int64  
 2   Hashed_deal_detail_1     3168 non-null   float64
 3   Hashed_deal_detail_2     3168 non-null   int64  
 4   Hashed_deal_detail_3     3168 non-null   int64  
 5   Hashed_deal_detail_4     3168 non-null   float64
 6   Hashed_deal_detail_5     3168 non-null   int64  
 7   Age                      3168 non-null   float64
 8   Gender                   3168 non-null   int64  
 9   Tavern_district_1        3168 non-null   int64  
 10  Tavern_district_2        3168 non-null   int64  
 11  Tavern_district_3        3168 non-null   int64  
 12  Tavern_district_4        3168 non-null   int64  
 13  Tavern_district_5        3168 non-null   int64  
 14  Tavern_district_6       

In [19]:
cols = X.columns

X = X[cols].astype({'Successful_deals_count' : "int",
                      'Tavern' : "int",
                      'Hashed_deal_detail_1' : "int",
                      'Hashed_deal_detail_2' : "int",
                      'Hashed_deal_detail_3' : "int",
                      'Hashed_deal_detail_4' : "int",
                      'Hashed_deal_detail_5' : "int",
                      'Age' : "int",
                      'Gender' : "int",
                      'Tavern_district_1' : "int",
                      'Tavern_district_2' : "int",
                      'Tavern_district_3' : "int",
                      'Tavern_district_4' : "int",
                      'Tavern_district_5' : "int",
                      'Tavern_district_6' : "int",
                      'Tavern_district_7' : "int",
                      'day_before_first_defolt' : "int",
                      'Deal_day' : "int",
                      'Deal_month' : "int",
                      'is_weekend' : "int"
                     })

# Catboost

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=2023)

In [21]:
X_train.columns

Index(['Successful_deals_count', 'Tavern', 'Hashed_deal_detail_1',
       'Hashed_deal_detail_2', 'Hashed_deal_detail_3', 'Hashed_deal_detail_4',
       'Hashed_deal_detail_5', 'Age', 'Gender', 'Tavern_district_1',
       'Tavern_district_2', 'Tavern_district_3', 'Tavern_district_4',
       'Tavern_district_5', 'Tavern_district_6', 'Tavern_district_7',
       'day_before_first_defolt', 'Deal_day', 'Deal_month', 'is_weekend'],
      dtype='object')

In [22]:
cat_cols = [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19]

# Successful_deals_count, 
# Tavern, 
# Hashed_deal_detail_1, 
# Hashed_deal_detail_2, 
# Hashed_deal_detail_3
# Hashed_deal_detail_4
# Hashed_deal_detail_5
# Gender
# Tavern_district_1
# Tavern_district_2 
# Tavern_district_3,
# Tavern_district_4, 
# Tavern_district_5, 
# Tavern_district_6, 
# Tavern_district_7
# Deal_day
# Deal_month
# is_weekend

In [23]:
train_pool = Pool(X_train, label=y_train, cat_features=cat_cols)
test_pool = Pool(X_test, label=y_test, cat_features=cat_cols)

In [24]:
clf = CatBoostClassifier(l2_leaf_reg=10, depth=5, learning_rate=0.9, iterations=1000, random_seed=2023)
clf.fit(train_pool)
y_pred = clf.predict(test_pool)

0:	learn: 0.3317183	total: 171ms	remaining: 2m 50s
1:	learn: 0.3103532	total: 183ms	remaining: 1m 31s
2:	learn: 0.3089231	total: 193ms	remaining: 1m 4s
3:	learn: 0.3035440	total: 207ms	remaining: 51.6s
4:	learn: 0.2997671	total: 220ms	remaining: 43.8s
5:	learn: 0.2996739	total: 227ms	remaining: 37.6s
6:	learn: 0.2937983	total: 240ms	remaining: 34.1s
7:	learn: 0.2913234	total: 253ms	remaining: 31.3s
8:	learn: 0.2910721	total: 259ms	remaining: 28.5s
9:	learn: 0.2891813	total: 270ms	remaining: 26.8s
10:	learn: 0.2833866	total: 282ms	remaining: 25.4s
11:	learn: 0.2813612	total: 294ms	remaining: 24.2s
12:	learn: 0.2772167	total: 306ms	remaining: 23.3s
13:	learn: 0.2768550	total: 319ms	remaining: 22.5s
14:	learn: 0.2761287	total: 332ms	remaining: 21.8s
15:	learn: 0.2723816	total: 344ms	remaining: 21.2s
16:	learn: 0.2713381	total: 356ms	remaining: 20.6s
17:	learn: 0.2672498	total: 368ms	remaining: 20.1s
18:	learn: 0.2619911	total: 379ms	remaining: 19.6s
19:	learn: 0.2612166	total: 391ms	remai

In [25]:
roc_auc_score(y_test, y_pred)

0.5078125

In [26]:
precision_score(y_test, y_pred)

0.16

In [27]:
recall_score(y_test, y_pred)

0.045454545454545456

In [28]:
clf.feature_importances_

array([ 7.63583276, 10.78810527,  0.46780864,  3.23952416, 11.28768555,
        2.45734844,  0.52025124, 11.96328704,  2.90823019,  0.40688267,
        0.92931399,  1.39419203,  1.44814751,  0.88213257,  0.65341033,
        0.26976926, 12.82575815, 14.20996111, 13.43961478,  2.27274432])

In [29]:
# Отметим важные факторы с текущим датасетом
importances = clf.feature_importances_
names = train_pool.get_feature_names()

named_importances = [[importances[i], names[i]] for i in range(len(names))]
print(np.array(sorted(named_importances, reverse=True)))

[['14.209961113447669' 'Deal_day']
 ['13.439614781695859' 'Deal_month']
 ['12.825758152158762' 'day_before_first_defolt']
 ['11.963287040226062' 'Age']
 ['11.287685546552279' 'Hashed_deal_detail_3']
 ['10.78810526520468' 'Tavern']
 ['7.635832762505826' 'Successful_deals_count']
 ['3.2395241562575268' 'Hashed_deal_detail_2']
 ['2.9082301865241162' 'Gender']
 ['2.4573484434108677' 'Hashed_deal_detail_4']
 ['2.272744316549271' 'is_weekend']
 ['1.4481475059556959' 'Tavern_district_4']
 ['1.3941920326384794' 'Tavern_district_3']
 ['0.9293139946523905' 'Tavern_district_2']
 ['0.8821325651699651' 'Tavern_district_5']
 ['0.6534103337199725' 'Tavern_district_6']
 ['0.5202512353688001' 'Hashed_deal_detail_5']
 ['0.46780863797153555' 'Hashed_deal_detail_1']
 ['0.4068826684140061' 'Tavern_district_1']
 ['0.2697692615761766' 'Tavern_district_7']]


Очень слабый recall. Нужно регулировать threshold. Также возможно в удаленных данных было что-то, что нам поможет. Как это узнать?

Посмотрим сколько заполненных ячеек в этих признаках для положительного таргета и все становится ясно.

In [30]:
dwarves = pd.read_csv("../data/raw/train.csv")

In [31]:
dwarves[dwarves['Default'] ==1]['Secret_dwarf_info_1'].value_counts().sum()

116

In [32]:
dwarves['Default'].value_counts()

0    2817
1     351
Name: Default, dtype: int64

In [33]:
# для 33% положительных таргетов содержится информация о фичах, которые мы в baseline удалили. Давайте исправлять.
(116 / 351) *100

33.04843304843305

Продолжение в ноутбуке HW_6_processed